In [2]:
import pandas as pd

# Opening the file
sample_tl_raw = open("src/text data/Bible_Tagalog.txt").read()

# Printing the Ilokano Raw Data
sample_tl_raw

"GENESIS\nNilalang ng Dios ang sanglibutan.\nNang pasimula ay nilikha ng Dios ang langit at ang lupa.\nAt ang lupa ay walang anyo at walang laman; at ang kadiliman ay sumasa ibabaw ng kalaliman; at ang Espiritu ng Dios ay sumasa ibabaw ng tubig.\nAt sinabi ng Dios Magkaroon ng liwanag; at nagkaroon ng liwanag.\nAt nakita ng Dios ang liwanag na mabuti, at inihiwalay ng Dios ang liwanag sa kadiliman.\nAt tinawag ng Dios ang liwanag na Araw, at tinawag niya ang kadiliman na Gabi. At nagkahapon at nagkaumaga ang unang araw.\nAt sinabi ng Dios, Magkaroon ng isang kalawakan sa gitna ng tubig, at mahiwalay ang tubig sa kapuwa tubig.\nAt ginawa ng Dios ang kalawakan, at inihiwalay ang tubig na nasa ilalim ng kalawakan, sa tubig na nasa itaas ng kalawakan: at nagkagayon.\nAt tinawag ng Dios ang kalawakan na Langit. At nagkahapon at nagkaumaga ang ikalawang araw.\nAt sinabi ng Dios, Mapisan ang tubig na nasa silong ng langit sa isang dako, at lumitaw ang katuyuan, at nagkagayon.\nAt tinawag ng D

In [3]:
# Splitting the raw data into sentences
parsed_sp_tl_raw = sample_tl_raw.split("\n")

parsed_sp_tl_raw 

['GENESIS',
 'Nilalang ng Dios ang sanglibutan.',
 'Nang pasimula ay nilikha ng Dios ang langit at ang lupa.',
 'At ang lupa ay walang anyo at walang laman; at ang kadiliman ay sumasa ibabaw ng kalaliman; at ang Espiritu ng Dios ay sumasa ibabaw ng tubig.',
 'At sinabi ng Dios Magkaroon ng liwanag; at nagkaroon ng liwanag.',
 'At nakita ng Dios ang liwanag na mabuti, at inihiwalay ng Dios ang liwanag sa kadiliman.',
 'At tinawag ng Dios ang liwanag na Araw, at tinawag niya ang kadiliman na Gabi. At nagkahapon at nagkaumaga ang unang araw.',
 'At sinabi ng Dios, Magkaroon ng isang kalawakan sa gitna ng tubig, at mahiwalay ang tubig sa kapuwa tubig.',
 'At ginawa ng Dios ang kalawakan, at inihiwalay ang tubig na nasa ilalim ng kalawakan, sa tubig na nasa itaas ng kalawakan: at nagkagayon.',
 'At tinawag ng Dios ang kalawakan na Langit. At nagkahapon at nagkaumaga ang ikalawang araw.',
 'At sinabi ng Dios, Mapisan ang tubig na nasa silong ng langit sa isang dako, at lumitaw ang katuyuan, 

In [4]:
dict_sm_tl = pd.DataFrame(parsed_sp_tl_raw, columns = ['Sentence'])

# Printing the first 5 rows of the DataFrame
dict_sm_tl.head()

,Sentence
0,GENESIS
1,Nilalang ng Dios ang sanglibutan.
2,Nang pasimula ay nilikha ng Dios ang langit at...
3,At ang lupa ay walang anyo at walang laman; at...
4,At sinabi ng Dios Magkaroon ng liwanag; at nag...


## Viterbi Algorithm

In [5]:
# Opening the determiners file
tl_determiners = open("src/text data/TL_Determiners.txt").read()

tl_determiners

'ng\nang\nnang\nay\nmga\nna\nsa'

In [6]:
# Parsing the determiners file
parsed_tl_dtmn = tl_determiners.split("\n")

parsed_tl_dtmn

['ng', 'ang', 'nang', 'ay', 'mga', 'na', 'sa']

## Cleaning the Data

In [7]:
import string

def remove_punct(pText):
    text_nopumct = "".join([char for char in pText if char not in string.punctuation])
    return text_nopumct

cleaned_sp_tl = [remove_punct(word) for word in parsed_sp_tl_raw]

cleaned_sp_tl[0:500]

['GENESIS',
 'Nilalang ng Dios ang sanglibutan',
 'Nang pasimula ay nilikha ng Dios ang langit at ang lupa',
 'At ang lupa ay walang anyo at walang laman at ang kadiliman ay sumasa ibabaw ng kalaliman at ang Espiritu ng Dios ay sumasa ibabaw ng tubig',
 'At sinabi ng Dios Magkaroon ng liwanag at nagkaroon ng liwanag',
 'At nakita ng Dios ang liwanag na mabuti at inihiwalay ng Dios ang liwanag sa kadiliman',
 'At tinawag ng Dios ang liwanag na Araw at tinawag niya ang kadiliman na Gabi At nagkahapon at nagkaumaga ang unang araw',
 'At sinabi ng Dios Magkaroon ng isang kalawakan sa gitna ng tubig at mahiwalay ang tubig sa kapuwa tubig',
 'At ginawa ng Dios ang kalawakan at inihiwalay ang tubig na nasa ilalim ng kalawakan sa tubig na nasa itaas ng kalawakan at nagkagayon',
 'At tinawag ng Dios ang kalawakan na Langit At nagkahapon at nagkaumaga ang ikalawang araw',
 'At sinabi ng Dios Mapisan ang tubig na nasa silong ng langit sa isang dako at lumitaw ang katuyuan at nagkagayon',
 'At tin

In [8]:
import re

def tokenize(text):
    tokens = re.split('\W+', text.lower())
    return tokens


tokenized_sp_tl = [tokenize(word) for word in parsed_sp_tl_raw]

dict_sm_tl['Tokenized'] = tokenized_sp_tl
dict_sm_tl.head()

,Sentence,Tokenized
0,GENESIS,[genesis]
1,Nilalang ng Dios ang sanglibutan.,"[nilalang, ng, dios, ang, sanglibutan, ]"
2,Nang pasimula ay nilikha ng Dios ang langit at...,"[nang, pasimula, ay, nilikha, ng, dios, ang, l..."
3,At ang lupa ay walang anyo at walang laman; at...,"[at, ang, lupa, ay, walang, anyo, at, walang, ..."
4,At sinabi ng Dios Magkaroon ng liwanag; at nag...,"[at, sinabi, ng, dios, magkaroon, ng, liwanag,..."


In [9]:
def check_dtmn(sentence, dtmn_list):
    """
    This function checks if the specific word in the sentence is a determiner, and extracts it.
    """
    text = [word for word in sentence if word in dtmn_list]
    
    return text


dict_sm_tl['Determiner'] = dict_sm_tl['Tokenized'].apply(lambda x: check_dtmn(x, parsed_tl_dtmn))

dict_sm_tl.head()

,Sentence,Tokenized,Determiner
0,GENESIS,[genesis],[]
1,Nilalang ng Dios ang sanglibutan.,"[nilalang, ng, dios, ang, sanglibutan, ]","[ng, ang]"
2,Nang pasimula ay nilikha ng Dios ang langit at...,"[nang, pasimula, ay, nilikha, ng, dios, ang, l...","[nang, ay, ng, ang, ang]"
3,At ang lupa ay walang anyo at walang laman; at...,"[at, ang, lupa, ay, walang, anyo, at, walang, ...","[ang, ay, ang, ay, ng, ang, ng, ay, ng]"
4,At sinabi ng Dios Magkaroon ng liwanag; at nag...,"[at, sinabi, ng, dios, magkaroon, ng, liwanag,...","[ng, ng, ng]"


In [31]:
""" 
    Affixes
"""
PREFIX_SET = [
    'nakikipag', 'pakikipag',
    'pinakama', 'pagpapa',
    'pinagka', 'panganga',
    'makapag', 'nakapag',
    'tagapag', 'makipag',
    'nakipag', 'tigapag',
    'pakiki', 'magpa',
    'napaka', 'pinaka',
    'ipinag', 'pagka',
    'pinag', 'mapag',
    'mapa', 'taga',
    'ipag', 'tiga',
    'pala', 'pina',
    'pang', 'naka',
    'nang', 'mang',
    'sing',
    'ipa', 'pam',
    'pan', 'pag',
    'tag', 'mai',
    'mag', 'nam',
    'nag', 'man',
    'may', 'ma',
    'na', 'ni',
    'pa', 'ka',
    'um', 'in',
    'i',
]

INFIX_SET = [
    'um', 'in',
]

SUFFIX_SET = [
    'syon','dor',
    'ita', 'han',
    'hin', 'ing',
    'ang', 'ng',
    'an', 'in',
    'g',
]

PREPO_SET = [
    "sumasa", 'gitna',
    'ibabaw', 'ilalim',
    'itaas', 'ibabaw',
]

PER_PRONOUN = [
    'ako', 'ikaw', 'siya', 'kami', 'kayo', 'sila',
    'ko', 'akin', 'sakin', 'amin', 'atin', 'inyo',
    'kata', 'mo', 'kanila', 'kanya', 'namin', 'natin', 
    'ninyo', 'niya'
]

noun_dtmn_list = ["ang", "ng", "mga"]

adv_dtmn_list = ["nang"]

prepo_dtmn_list = ["sa"]



## Verb tagger

In [18]:
def tag_verb(sentence, prefix_list, infix_list, suffix_list, dtmn_list, prepo_list, pronoun_list):
    """
    This function tags if the specific word in the sentence is a verb, and extracts it.
    """
    
    """
    verb = [word for word in sentence if word in verb_list]
    """
    verb = []
    prev_word = ""
              
    for word in sentence:
        isDone = False
                
        try:
            next_word = sentence[sentence.index(word) + 1]
        except (ValueError, IndexError):
            next_word = None
                        
        if word not in (dtmn_list + prepo_list + pronoun_list):
            if prev_word not in (noun_dtmn_list + adv_dtmn_list + prepo_dtmn_list): 
                if next_word in noun_dtmn_list: 
                    for prefix in prefix_list:
                        if word.startswith(prefix) and not isDone:
                            verb.append(word)
                            isDone = True
                    for infix in infix_list:
                        if word.__contains__(infix) and not isDone:
                            verb.append(word)
                            isDone = True
                    for suffix in suffix_list:
                        """
                        words ending with 'ang' are adverbs and after the adverbs are the nouns 
                        """
                        if word.endswith(suffix) and not isDone and not word.endswith("ang") and not prev_word.endswith("ang"):
                            verb.append(word)
                            isDone = True
               
        prev_word = word
        
    return verb
# end of function

dict_sm_tl['Verb'] = dict_sm_tl['Tokenized'].apply(lambda x: tag_verb(x, PREFIX_SET, INFIX_SET, SUFFIX_SET, parsed_tl_dtmn, PREPO_SET, PER_PRONOUN))
dict_sm_tl.head(30)

,Sentence,Tokenized,Determiner,Verb
0,GENESIS,[genesis],[],[]
1,Nilalang ng Dios ang sanglibutan.,"[nilalang, ng, dios, ang, sanglibutan, ]","[ng, ang]",[nilalang]
2,Nang pasimula ay nilikha ng Dios ang langit at...,"[nang, pasimula, ay, nilikha, ng, dios, ang, l...","[nang, ay, ng, ang, ang]",[nilikha]
3,At ang lupa ay walang anyo at walang laman; at...,"[at, ang, lupa, ay, walang, anyo, at, walang, ...","[ang, ay, ang, ay, ng, ang, ng, ay, ng]",[]
4,At sinabi ng Dios Magkaroon ng liwanag; at nag...,"[at, sinabi, ng, dios, magkaroon, ng, liwanag,...","[ng, ng, ng]","[sinabi, magkaroon, nagkaroon]"
5,"At nakita ng Dios ang liwanag na mabuti, at in...","[at, nakita, ng, dios, ang, liwanag, na, mabut...","[ng, ang, na, ng, ang, sa]","[nakita, inihiwalay]"
6,"At tinawag ng Dios ang liwanag na Araw, at tin...","[at, tinawag, ng, dios, ang, liwanag, na, araw...","[ng, ang, na, ang, na, ang]","[tinawag, tinawag, nagkaumaga]"
7,"At sinabi ng Dios, Magkaroon ng isang kalawaka...","[at, sinabi, ng, dios, magkaroon, ng, isang, k...","[ng, ng, sa, ng, ang, sa]","[sinabi, magkaroon, mahiwalay]"
8,"At ginawa ng Dios ang kalawakan, at inihiwalay...","[at, ginawa, ng, dios, ang, kalawakan, at, ini...","[ng, ang, ang, na, ng, sa, na, ng]","[ginawa, inihiwalay]"
9,At tinawag ng Dios ang kalawakan na Langit. At...,"[at, tinawag, ng, dios, ang, kalawakan, na, la...","[ng, ang, na, ang]","[tinawag, nagkaumaga]"


In [12]:
temp_word = "maganda"

temp_len = len(temp_word.removeprefix("mag"))

print(temp_len)

4


In [44]:
def tag_noun(sentence, per_noun_list):
    """
    This function tags if the specific word in the sentence is a noun, and extracts it.
    """
    noun = []
    prev_word = ""
    prev2_word = ""
    adj_prefix = ["ika", "pinaka"]
    
    
    for word in sentence:
        # print(word)
        """
        marks if the word is already tagged
        """
        isDone = False
        
        """
        getting the next word
        """
        try:
            next_word = sentence[sentence.index(word) + 1]
        except (ValueError, IndexError):
            next_word = None
        
        
        if word in per_noun_list:
            noun.append(word)
            isDone = True
        
        if prev_word in noun_dtmn_list and word not in noun_dtmn_list and not isDone:
            for prefix in adj_prefix:
                if (word.startswith(prefix)):
                    isDone = True
                else:
                    noun.append(word)
                    isDone = True
            
        """
        For words after the adverb
        """
        if prev2_word == "ay" and prev_word.endswith("ang") and word not in noun_dtmn_list and not isDone:
            noun.append(word)
            isDone = True
        
        
        """
        getting the previous word
        """
        prev_word = word
        
        
        
        """
        getting the previous after the previous word
        """
        try:
            prev2_word = sentence[sentence.index(word) - 1]
        except (ValueError, IndexError):
            prev2_word = None
        
    return noun
# end of function
        
        
dict_sm_tl['Noun'] = dict_sm_tl['Tokenized'].apply(lambda x: tag_noun(x, PER_PRONOUN))
dict_sm_tl.head(30)
    

dios
dios
sanglibutan
sanglibutan
dios
dios
langit
langit
lupa
lupa
lupa
lupa
kadiliman
kadiliman
kalaliman
kalaliman
espiritu
espiritu
dios
dios
tubig
tubig
dios
dios
liwanag
liwanag
liwanag
liwanag
dios
dios
liwanag
liwanag
dios
dios
liwanag
liwanag
dios
dios
liwanag
liwanag
kadiliman
kadiliman
unang
unang
dios
dios
isang
isang
tubig
tubig
tubig
tubig
dios
dios
kalawakan
kalawakan
tubig
tubig
kalawakan
kalawakan
kalawakan
kalawakan
dios
dios
kalawakan
kalawakan
ikalawang
ikalawang
dios
dios
tubig
tubig
langit
langit
katuyuan
katuyuan
dios
dios
katuyuan
katuyuan
kapisanan
kapisanan
tubig
tubig
dagat
dagat
dios
dios
dios
dios
lupa
lupa
damo
damo
kaniyang
kaniyang
lupa
lupa
lupa
lupa
damo
damo
punong
punong
kaniyang
kaniyang
dios
dios
ikatlong
ikatlong
dios
dios
tanglaw
tanglaw
langit
langit
araw
araw
panahon
panahon
araw
araw
taon
taon
langit
langit
lupa
lupa
dios
dios
dalawang
dalawang
malaking
malaking
maliit
maliit
bituin
bituin
inilagay
inilagay
dios
dios
langit
langit
lupa
lupa
li

,Sentence,Tokenized,Determiner,Verb,Noun
0,GENESIS,[genesis],[],[],[]
1,Nilalang ng Dios ang sanglibutan.,"[nilalang, ng, dios, ang, sanglibutan, ]","[ng, ang]",[nilalang],"[dios, sanglibutan]"
2,Nang pasimula ay nilikha ng Dios ang langit at...,"[nang, pasimula, ay, nilikha, ng, dios, ang, l...","[nang, ay, ng, ang, ang]",[nilikha],"[dios, langit, lupa]"
3,At ang lupa ay walang anyo at walang laman; at...,"[at, ang, lupa, ay, walang, anyo, at, walang, ...","[ang, ay, ang, ay, ng, ang, ng, ay, ng]",[],"[lupa, anyo, laman, kadiliman, kalaliman, espi..."
4,At sinabi ng Dios Magkaroon ng liwanag; at nag...,"[at, sinabi, ng, dios, magkaroon, ng, liwanag,...","[ng, ng, ng]","[sinabi, magkaroon, nagkaroon]","[dios, liwanag, liwanag]"
5,"At nakita ng Dios ang liwanag na mabuti, at in...","[at, nakita, ng, dios, ang, liwanag, na, mabut...","[ng, ang, na, ng, ang, sa]","[nakita, inihiwalay]","[dios, liwanag, dios, liwanag]"
6,"At tinawag ng Dios ang liwanag na Araw, at tin...","[at, tinawag, ng, dios, ang, liwanag, na, araw...","[ng, ang, na, ang, na, ang]","[tinawag, tinawag, nagkaumaga]","[dios, liwanag, niya, kadiliman, unang]"
7,"At sinabi ng Dios, Magkaroon ng isang kalawaka...","[at, sinabi, ng, dios, magkaroon, ng, isang, k...","[ng, ng, sa, ng, ang, sa]","[sinabi, magkaroon, mahiwalay]","[dios, isang, tubig, tubig]"
8,"At ginawa ng Dios ang kalawakan, at inihiwalay...","[at, ginawa, ng, dios, ang, kalawakan, at, ini...","[ng, ang, ang, na, ng, sa, na, ng]","[ginawa, inihiwalay]","[dios, kalawakan, tubig, kalawakan, kalawakan]"
9,At tinawag ng Dios ang kalawakan na Langit. At...,"[at, tinawag, ng, dios, ang, kalawakan, na, la...","[ng, ang, na, ang]","[tinawag, nagkaumaga]","[dios, kalawakan, ikalawang]"
